In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from sklearn import model_selection, svm
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
import data_process_funcs as dpf
import ml_funcs as mlf

In [ ]:
#load data
nr_qubits =4
df_4q_R = dpf.get_expanded_df('Refreshed_Simulation',nr_qubits)
#df_4q_H = dpf.get_expanded_df('Hardware',nr_qubits)
#df_4q_S = dpf.get_expanded_df('Simulation',nr_qubits)

In [ ]:
def get_HSR_array_with_same_backends(nr_qubits):
    df_R = dpf.get_expanded_df('Refreshed_Simulation',nr_qubits)
    df_H = dpf.get_expanded_df('Hardware',nr_qubits)
    df_S = dpf.get_expanded_df('Simulation',nr_qubits)

    df_S = df_S[df_S['backend']=='torino'| df_S['backend']=='brisbane']
    
    return [df_H, df_S, df_R]

def get_circuit_type_array(df_nq):
    df_nqi = mlf.features_to_int(df_nq)
    circuits = mlf.split_into_circuits(df_nqi) 
    return circuits

def get_HSR_list_of_arrays(nr_qubits):
    initial_list = get_HSR_array_with_same_backends(nr_qubits)
    list_of_arrays = mlf.generate_combos(initial_list)

    #DO WE WANT TO USE TRAIN ON H, TEST ON SR COMBINED???
    
    train_SR_test_H = [pd.concat(initial_list[1,2]),initial_list[0]]
    # NEED  TO ADJUST CREATE_CSV FOR THIS^
    
    list_of_arrays.append(train_SR_test_H)

    return list_of_arrays

def get_circuits_list_of_arrays(df_nq):
    circuits = get_circuit_type_array(df_nq)
    combos = mlf.generate_combos(circuits,True)
    #[[df1, df2, df3,pd.concat(df2,df3)],
    #[df2, df1, df3,pd.concat(df1,df3)],
    # df3, df1, df2,pd.concat(df1,df2)]]
    for i in len(circuits):
        combined_train = [combos[i][4],combos[i][0]]
        combos.append(combined_train)
    
    return combos
